In [5]:
import numpy as np
import pandas as pd

In [6]:
paper_cs = pd.read_csv('computer_science_paper_subset.csv', index_col = 0)
paper_is = pd.read_csv('information_science_paper_subset.csv', index_col = 0)

/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## CS

In [7]:
chunks=pd.read_csv('./All data/advanced/PaperFieldsOfStudy.txt',chunksize=1000000,sep='\t', header = None,\
                   names = ['PaperId', 'FieldOfStudyId', 'Score'])

In [8]:
%%time
paper_topic = pd.DataFrame()

i = 0
for chunk in chunks:
    paper_topic = pd.concat([paper_topic, chunk[chunk['PaperId'].isin(paper_cs['PaperId'].unique())]])
    i = i+1
    if i % 100 == 0:
        print(i)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
CPU times: user 9min 35s, sys: 51.4 s, total: 10min 26s
Wall time: 10min 26s


In [9]:
computer_science_conference = paper_topic[['PaperId', 'FieldOfStudyId']].merge(paper_cs[['PaperId', 'ConferenceSeriesId', 'Year',\
                                                                                         'Date']], on = 'PaperId')

In [10]:
computer_science_conference.rename(columns = {'PaperId' : 'article', 'FieldOfStudyId' : 'topic',\
                                              'ConferenceSeriesId' : 'conference', 'Year' : 'year', 'Date' : 'date'},\
                                   inplace = True)

In [ ]:
computer_science_conference.to_csv('computer_science_conference_subset.csv')

In [11]:
topics=pd.read_csv('./All data/advanced/FieldsOfStudy.txt',sep='\t', header = None,\
                   names = ['topic', 'Rank', 'NormalizedName', 'name', 'MainType', 'Level', 'PaperCount',\
                            'CitationCount', 'CreatedDate'])

In [12]:
topics.head()

,topic,Rank,NormalizedName,name,MainType,Level,PaperCount,CitationCount,CreatedDate
0,417682,20658,night air,Night air,NaN,2,96,504,2016-06-24
1,1443462,18729,immobiliser,Immobiliser,NaN,2,400,1057,2016-06-24
2,1576492,13111,matrix pencil,Matrix pencil,NaN,3,1708,24700,2016-06-24
3,2657588,14817,combinatorial topology,Combinatorial topology,NaN,3,528,13987,2016-06-24
4,3079626,9292,quantum electrodynamics,Quantum electrodynamics,NaN,1,425305,6469899,2016-06-24


In [13]:
cs_data = computer_science_conference.copy()
cs_data.drop(columns=['date'],inplace=True)
cs_data.sort_values(by='year',inplace=True)

In [14]:
##Remain recent years and somewhat frequent journal/topic.
cs_data = cs_data[cs_data.year>=1990]
# conference_count = cs_data .groupby('conference').count().reset_index()
# conference_list = conference_count.conference [  conference_count.article>=1000].unique()
# topic_count = cs_data.groupby('topic').count().reset_index()
# topic_list = topic_count.topic[  topic_count.article>=500].unique()
# year_list = cs_data .year.unique()

# cs_data = cs_data[cs_data.conference.isin(conference_list)]  
# cs_data = cs_data[cs_data.topic.isin(topic_list)]

In [15]:
#  Identify sorted keywords from 1990 to 2020
cs_data_top = cs_data.groupby('topic', as_index = False).count().sort_values('conference', ascending = False)\
                [['topic', 'conference']]
cs_data_top.reset_index(inplace = True, drop = True)
cs_data_top_name = cs_data_top.merge(topics[['topic', 'name', 'Level']], how = 'left', on = 'topic')
cs_data_top_name.rename(columns = {'topic' : 'topic_30_years', 'conference' : 'count_30_years', 'name' : 'name_30_years'}\
                          , inplace = True)

In [ ]:
cs_data_top_name.to_csv('Computer_Science_Conference_Subset_Top_Keywords.csv')

##  Level Keywords

In [16]:
cs_data_top_name

,topic_30_years,count_30_years,name_30_years,Level
0,41008148,360733,Computer science,0
1,154945302,208411,Artificial intelligence,1
2,119857082,86919,Machine learning,1
3,178980831,71794,Pattern recognition,1
4,31972630,53923,Computer vision,1
...,...,...,...,...
64367,2776112827,1,Digital morphogenesis,4
64368,2776113144,1,Double outlet right ventricle,4
64369,2776113583,1,Subprime lending,3
64370,2984214749,1,Multimedia metadata,4


In [36]:
keywords_level_top = cs_data_top_name.groupby(["Level"]).apply(lambda x: x.sort_values(["count_30_years"], ascending = False)[:100])

In [37]:
keywords_level_top.to_csv('Computer_Science_Conference_Subset_Top_Keywords_By_Level.csv')

## IS

In [38]:
chunks=pd.read_csv('./All data/advanced/PaperFieldsOfStudy.txt',chunksize=1000000,sep='\t', header = None,\
                   names = ['PaperId', 'FieldOfStudyId', 'Score'])

In [39]:
%%time
paper_topic = pd.DataFrame()

i = 0
for chunk in chunks:
    paper_topic = pd.concat([paper_topic, chunk[chunk['PaperId'].isin(paper_is['PaperId'].unique())]])
    i = i+1
    if i % 100 == 0:
        print(i)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
CPU times: user 5min 2s, sys: 10.3 s, total: 5min 12s
Wall time: 5min 12s


In [40]:
information_science_journal = paper_topic[['PaperId', 'FieldOfStudyId']].merge(paper_is[['PaperId', 'JournalId', 'Year',\
                                                                                         'Date']], on = 'PaperId')

In [41]:
information_science_journal.rename(columns = {'PaperId' : 'article', 'FieldOfStudyId' : 'topic',\
                                              'JournalId' : 'journal', 'Year' : 'year', 'Date' : 'date'},\
                                   inplace = True)

In [42]:
information_science_journal.to_csv('information_science_journal_subset.csv')

In [43]:
information_science_journal

,article,topic,journal,year,date
0,196357788,113993141,57293258.0,2014.0,2014-03-01
1,196357788,186855751,57293258.0,2014.0,2014-03-01
2,196357788,114611597,57293258.0,2014.0,2014-03-01
3,196357788,162853370,57293258.0,2014.0,2014-03-01
4,196357788,2777301405,57293258.0,2014.0,2014-03-01
...,...,...,...,...,...
260798,3000928323,162324750,33323087.0,1978.0,1978-11-01
260799,3000928323,195094911,33323087.0,1978.0,1978-11-01
260800,3000928323,175444787,33323087.0,1978.0,1978-11-01
260801,3000928323,155109394,33323087.0,1978.0,1978-11-01


In [44]:
topics=pd.read_csv('./All data/advanced/FieldsOfStudy.txt',sep='\t', header = None,\
                   names = ['topic', 'Rank', 'NormalizedName', 'name', 'MainType', 'Level', 'PaperCount',\
                            'CitationCount', 'CreatedDate'])

In [45]:
topics.head()

,topic,Rank,NormalizedName,name,MainType,Level,PaperCount,CitationCount,CreatedDate
0,417682,20658,night air,Night air,NaN,2,96,504,2016-06-24
1,1443462,18729,immobiliser,Immobiliser,NaN,2,400,1057,2016-06-24
2,1576492,13111,matrix pencil,Matrix pencil,NaN,3,1708,24700,2016-06-24
3,2657588,14817,combinatorial topology,Combinatorial topology,NaN,3,528,13987,2016-06-24
4,3079626,9292,quantum electrodynamics,Quantum electrodynamics,NaN,1,425305,6469899,2016-06-24


In [46]:
is_data = information_science_journal.copy()
is_data.drop(columns=['date'],inplace=True)
is_data.sort_values(by='year',inplace=True)

In [47]:
##Remain recent years and somewhat frequent journal/topic.
is_data = is_data[is_data.year>=1990]
# conference_count = cs_data .groupby('conference').count().reset_index()
# conference_list = conference_count.conference [  conference_count.article>=1000].unique()
# topic_count = cs_data.groupby('topic').count().reset_index()
# topic_list = topic_count.topic[  topic_count.article>=500].unique()
# year_list = cs_data .year.unique()

# cs_data = cs_data[cs_data.conference.isin(conference_list)]  
# cs_data = cs_data[cs_data.topic.isin(topic_list)]

In [48]:
#  Identify sorted keywords from 1990 to 2020
is_data_top = is_data.groupby('topic', as_index = False).count().sort_values('journal', ascending = False)\
                [['topic', 'journal']]
is_data_top.reset_index(inplace = True, drop = True)
is_data_top_name = is_data_top.merge(topics[['topic', 'name']], how = 'left', on = 'topic')
is_data_top_name.rename(columns = {'topic' : 'topic_30_years', 'journal' : 'count_30_years', 'name' : 'name_30_years'}\
                          , inplace = True)

In [49]:
is_data_top_name.to_csv('Information_Science_Journal_Subset_Top_Keywords.csv')